# Paso 0: Dependencias

In [ ]:
#!pip install smart-open
#ref: https://radimrehurek.com/gensim/
#!pip install --upgrade gensim   or conda install -c conda-forge gensim

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_parquet('PC_p.parquet.gzip')

In [ ]:
df.head()

In [ ]:
df.shape

### Construir diccionario de términos

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary(df.tokens)

In [ ]:
print(dictionary)

### Construir matriz de documentos vs términos

In [ ]:
# libreria para paralelizar
import multiprocessing as mp
import time

t0 = time.time()
pool = mp.Pool(mp.cpu_count()-3)
doc_term_matrix = pool.map(dictionary.doc2bow, [sentence for sentence in df.tokens])
pool.close()
print(time.time()-t0)

### Construir modelo LDA

In [ ]:
from gensim.models.ldamulticore import LdaMulticore

t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=50, id2word = dictionary, passes=10, workers=69)
print(time.time()-t0)

### Ejemplos de visualización del modelo

In [ ]:
# Mostrar los términos y sus pesos de un documento
print(list(lda_model[doc_term_matrix[0]]))

# Mostrar los términos más relevantes de los tópicos más relevantes tópico y sus pesos
print(lda_model.print_topics(num_topics=10, num_words=3))

## Visualización de todos los tópicos

In [ ]:
import pyLDAvis
from pyLDAvis import gensim

t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensim.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
print(time.time()-t0)
vis

### Guardar la visualización en un archivo HTML

In [ ]:
pyLDAvis.save_html(vis, 'lda_visualization.html')